
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
class ResidualUnit(keras.layers.Layer):
  def __init__(self, filters, strides, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.activation = keras.activations.get(activation)
    self.main_layers = [
    keras.layers.Conv2D(filters, 3, strides= strides, padding='SAME', 
                        use_bias= False),
    keras.layers.BatchNormalization(),
    self.activation,
    keras.layers.Conv2D(filters, 3, strides= 1, padding='SAME', 
                        use_bias= False),
    keras.layers.BatchNormalization(),
    ]
    ####
    #skip layers are 1x1 convulations to reduce input resolution when required
    ####
    self.skip_layers = [] 
    if strides > 1:
      self.skip_layers = [
      keras.layers.Conv2D(filters, 1, strides= strides, padding='SAME', 
                        use_bias= False),
      keras.layers.BatchNormalization(), 
      ]

  def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
    return self.activation(skip_Z + Z)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, strides=2, input_shape=[224, 224, 3], 
                              padding='SAME', use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='SAME'))
prev_filter= 64
for filter in [64]*3 + [128]*4 + [256]*6 + [512]*3:
  strides=1 if filter == prev_filter else 2
  model.add(ResidualUnit(filter, strides))
  prev_filter = filter
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 112, 112, 64)      9408      
_________________________________________________________________
batch_normalization_4 (Batch (None, 112, 112, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
residual_unit_1 (ResidualUni (None, 56, 56, 64)        74240     
_________________________________________________________________
residual_unit_2 (ResidualUni (None, 56, 56, 64)        74240     
_________________________________________________________________
residual_unit_3 (ResidualUni (None, 56, 56, 64)       

Credits: [Hands-On Machine Learning with Scikit-Learn, Keras and TensorFlow](https://www.amazon.com/Hands-Machine-Learning-Scikit-Learn-TensorFlow-dp-1492032646/dp/1492032646/ref=dp_ob_image_bkhttps://www.amazon.com/Hands-Machine-Learning-Scikit-Learn-TensorFlow-dp-1492032646/dp/1492032646/ref=dp_ob_image_bk)